In [ ]:
## import numpy as np
import random
import time

#plagarized code from book, mostly
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.layers = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for y, x in zip(sizes[1:], sizes[:-1])]
        
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def trainNetwork(self, games, eta, gamma, ep, start):
        for i in range(games):
            if i%10000==0:
                print("game: ", i)
            total = start
            while True:
                innum1 = np.zeros((start,1))
                index = total - 1
                innum1[index] = 1
                outnum1 = self.feedforward(innum1)
                if random.random()>ep:
                    move_choice = np.argmax(outnum1)
                else:
                    move_choice = random.randint(0,2)
                subtractor = move_choice + 1
                total -= subtractor
                #print(subtractor, total)
                reward = rewardFunction(total)
                if reward!=0:
                    expected_out = outnum1
                    expected_out[move_choice] = reward
                    self.update_NN(eta, innum1, expected_out)
                    break
                rand_choice = random.randint(1,3)
                total -= rand_choice
                #print("R", rand_choice, total)
                reward = rewardFunction(total, Q=False)
                if reward!=0:
                    expected_out = outnum1
                    expected_out[move_choice] = reward
                    self.update_NN(eta, innum1, expected_out)
                    break
                innum2 = np.zeros((start,1))
                index2 = total-1
                innum2[index2] = 1
                outnum2 = NN.feedforward(innum2)
                maxQ = max(outnum2)
                expected_out = outnum1
                expected_out[move_choice] = gamma*maxQ
                self.update_NN(eta, innum1, expected_out)
                
    def update_NN(self, eta, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)
        nabla_b = [(nb + dnb) for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [(nw + dnw) for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-eta*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-eta*nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        #feedforward
        activation = x
        activations = [x]
        zs = []
        p = 0
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activations[p]) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            p += 1
        #backward pass
        delta = (activations[-1] - y)*sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, np.transpose(activations[-2]))
        for L in range(2, self.num_layers):
            z = zs[-L]
            sp = sigmoid_prime(z)
            delta = np.dot(np.transpose(self.weights[-L+1]), delta)*sp
            nabla_b[-L] = delta
            nabla_w[-L] = np.dot(delta, np.transpose(activations[-L-1]))
        return(nabla_b, nabla_w)
        
        
    def cost_derivative(a, y):
        return (a - y)
    
    
    def play(self, start, games):
        for i in range(games):
            total = start -1
            print("Total: ", total+1,"\n")
            while True:
                innum = np.zeros((21,1))
                innum[total] = 1
                choice = np.argmax(NN.feedforward(innum))
                total = total - choice -1
                print("Computer chooses: ",choice+1," Total remining: ",total+1,"\n")
                if checkGameOver(total):
                    print("Computer Wins!!!\n")
                    break
                while True:
                    pchoice = int(input("Player Select 1, 2, or 3: "))
                    if((pchoice != 1 and pchoice != 2 and pchoice !=3) or total-pchoice < -1):
                        print("Invalid Entry, Please select a legal move")
                    else:
                        break
                total = total - pchoice
                print("Total remaining: ",total+1,"\n")
                if checkGameOver(total):
                    print("Player Wins!!!\n")
                    break

    
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def checkGameOver(total):
    if total < 0:
        return True
    else:
        return False

def rewardFunction(number, Q=True):
    if Q:
        if number==0:
            return 10
        if number<0:
            return -1
    else:
        if number<=0:
            return -10
    return 0
#################################################################################################################
NN = Network([21,10,3])

start = time.time()
NN.trainNetwork(150000, 0.1, 1, 0.1, 21)  #convergence takes about 150,000 training games with learning rate 0.5
end = time.time()
print("Time: ", end - start)

NN.play(21, 3)